# Register TSV Data With Athena

In [71]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [72]:
ingest_create_athena_table_csv_passed = False

In [73]:
%store -r ingest_create_athena_db_passed

In [74]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [75]:
print(ingest_create_athena_db_passed)

True


In [76]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [77]:
%store -r s3_private_path_csv

In [78]:
try:
    s3_private_path_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [79]:
print(s3_private_path_csv)

s3://sagemaker-us-east-1-339129315232/Assignment_2


# Import PyAthena

In [80]:
from pyathena import connect

#### Dataset columns

- `Track ID`: ID for the specific track
- `artists`: Name of the band/artist that wrote the track
- `track_name`: name of the song.
- `popularity`: rating.
- `duration_ms`: Song length in MS.
- `explicit`: Explicit version true or false
- `dancability`: likely to make you dance.
- `energy`:
- `key`: 
- `loudness`:
- `mode`: 
- `speechiness`: 
- `acousticness`: 
- `instrumentalness`: 
- `liveness`:
- `valence`:
- `tempo`:
- `time_signature`:
- `track_genre`:


In [81]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [82]:
# Set Athena parameters
database_name = "dsoaws1"
table_name_csv = "cleaned_data_csv"

In [83]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [84]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    track_id VARCHAR(255),
    artists VARCHAR(255),
    track_name VARCHAR(255),
    popularity INT,
    duration_ms INT,
    explicit BOOLEAN,
    danceability FLOAT,
    energy FLOAT,
    key INT,
    loudness FLOAT,
    mode INT,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT,
    time_signature INT,
    track_genre VARCHAR(255)
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws1.cleaned_data_csv(
    track_id VARCHAR(255),
    artists VARCHAR(255),
    track_name VARCHAR(255),
    popularity INT,
    duration_ms INT,
    explicit BOOLEAN,
    danceability FLOAT,
    energy FLOAT,
    key INT,
    loudness FLOAT,
    mode INT,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT,
    time_signature INT,
    track_genre VARCHAR(255)
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-339129315232/Assignment_2'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [85]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_123/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [86]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_123/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,cleaned_data_csv


In [87]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_csv_passed = True

In [88]:
%store ingest_create_athena_table_csv_passed

Stored 'ingest_create_athena_table_csv_passed' (bool)


In [89]:
# List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99
statement = f"""
SELECT artists, track_name, popularity  
FROM {database_name}.{table_name_csv}
WHERE popularity > 98
"""

# Execute the query and store the result in a DataFrame
df = pd.read_sql(statement, conn)

# Display the first 5 rows of the result
print(df.head(5))

/tmp/ipykernel_123/49453657.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


                artists                 track_name  popularity
0  Sam Smith;Kim Petras  Unholy (feat. Kim Petras)         100
1  Sam Smith;Kim Petras  Unholy (feat. Kim Petras)         100


In [90]:
# List artists with an average popularity of 92
query_2 = f"""
SELECT artists, AVG(popularity) as avg_popularity
FROM {database_name}.{table_name_csv}
GROUP BY artists
HAVING AVG(popularity) = 92
"""

# Execute the query and store the result in a DataFrame
df_2 = pd.read_sql(query_2, conn)

# Display the result
print(df_2)

/tmp/ipykernel_123/379252283.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(query_2, conn)


             artists  avg_popularity
0       Harry Styles            92.0
1  Rema;Selena Gomez            92.0


In [113]:
# List the Top 10 most energetic genres
query_3 = f"""
SELECT
    track_genre,
    avg(energy) AS avg_energy
FROM {database_name}.{table_name_csv}
GROUP BY track_genre
ORDER BY avg_energy 
LIMIT 10
"""

# Execute the query and store the result in a DataFrame
df_3 = pd.read_sql(query_3, conn)

# Display the result
print(df_3)

/tmp/ipykernel_123/104331639.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3 = pd.read_sql(query_3, conn)


  track_genre  avg_energy
0       0.688    0.000000
1       0.227    0.000000
2           0    0.000000
3     new-age    0.217076
4     ambient    0.238977
5       0.224    0.250000
6     romance    0.300693
7      disney    0.302719
8       piano    0.315877
9      guitar    0.325546


In [34]:
# How many tracks is Bad Bunny on:
query_4 = f"""
SELECT
    COUNT(*) AS track_count
FROM {database_name}.{table_name_csv}
WHERE artists LIKE '%Bad Bunny%';
"""

# Execute the query and store the result in a DataFrame
df_4 = pd.read_sql(query_4, conn)

# Display the result
print(df_4)

/tmp/ipykernel_123/3849218040.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4 = pd.read_sql(query_4, conn)


   track_count
0          412


In [115]:
# Show the top 10 genres in terms of popularity sorted by their most popular track:
query_5 = f"""
SELECT
    track_genre AS genre,
    MAX(popularity) AS max_popularity,
    MAX(track_name) AS most_popular_track
FROM {database_name}.{table_name_csv}
GROUP BY track_genre
ORDER BY max_popularity DESC
LIMIT 10;
"""

# Execute the query and store the result in a DataFrame
df_5 = pd.read_sql(query_5, conn)

# Display the result
print(df_5)

/tmp/ipykernel_123/3230666881.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5 = pd.read_sql(query_5, conn)


       genre  max_popularity                  most_popular_track
0      dance             100                          ブラッディー・マリー
1        pop             100                 ダンシング・ウィズ・ア・ストレンジャー
2      latin              98  Échame La Culpa - Not On You Remix
3  reggaeton              98                      ¿Qué Nos Pasó?
4     latino              98                      cómo dormiste?
5        edm              98                        you heard me
6     reggae              98                                韻波句徒
7      piano              96               원펀맨 Theme - Sad Theme
8       rock              96                                 紅蓮華
9      chill              93                           your love


In [116]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_csv_passed                 -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
s3_private_path_csv                                   -> 's3://sagemaker-us-east-1-339129315232/Assignment_
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-339129315232/amazon-revi
s3_public_path_csv                                    -> 's3://dsoaws1/Assignment_2/'
s3_public_path_tsv                                    -> 's3://dsoaws/amazon-reviews-pds/tsv'
setup_dependencies_passed                             -> True
setup_s3_bucket_passed                                -> True


In [117]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [92]:
# Show the top 10 genres in terms of popularity sorted by their most popular track:
query_full = f"""
SELECT
    *
FROM dsoaws1.cleaned_data_csv;
"""

# Execute the query and store the result in a DataFrame
df_full = pd.read_sql(query_full, conn)

# Display the result
print(df_full)

/tmp/ipykernel_123/1984789438.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_full = pd.read_sql(query_full, conn)


                      track_id                 artists  \
0       5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1       4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2       1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3       6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4       5vjLSffimiIP26QG5WcN2K        Chord Overstreet   
...                        ...                     ...   
113544  2C3TZjDRiAzdyViavDJ217           Rainy Lullaby   
113545  1hIz5L4IB9hN3WRYPOCGPw           Rainy Lullaby   
113546  6x8ZfSoqDjuNa5SVP5QjvX           Cesária Evora   
113547  2e6sXL2bYv4bSz6VTdnfLs        Michael W. Smith   
113548  2hETkH7cOfqmz3LqZDHZf5           Cesária Evora   

                        track_name  popularity  duration_ms explicit  \
0                           Comedy        73.0     230666.0    False   
1                 Ghost - Acoustic        55.0     149610.0    False   
2                   To Begin Again        57.0     210826.0    False   
3       Can't H